<a href="https://colab.research.google.com/github/Rgalitzdorfer/Movie-Recommendation-System-/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#Import libraries
import numpy as np #Linear Algebra
import pandas as pd #Read dataframe
from sklearn.feature_extraction.text import TfidfVectorizer #Vectorizer
from sklearn.metrics.pairwise import cosine_similarity #Similarity Score
from sklearn.model_selection import train_test_split #Split train and test sets

1. **Data Collection**

In [2]:
#Import data
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)
data.shape

Saving movies.csv to movies.csv


(4803, 24)

In [14]:
#Display data
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


2. **Data Cleaning**

In [16]:
#Replace null values
#Select columns of importance
useful_columns = ['genres', 'keywords', 'id', 'original_title', 'tagline', 'original_language']

#Replace nulls
for column in useful_columns:
  data[column] = data[column].fillna('')

data.isnull().sum() #Check nulls

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    0
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [17]:
#Create new dataframe with columns for prediction
new_data = data[useful_columns].copy()
new_data.head()

,genres,keywords,id,original_title,tagline,original_language
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,19995,Avatar,Enter the World of Pandora.,en
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,285,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.",en
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,206647,Spectre,A Plan No One Escapes,en
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,49026,The Dark Knight Rises,The Legend Ends,en
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,49529,John Carter,"Lost in our world, found in another.",en


3. **Model Training**

In [6]:
#Separate train and test data
train_data, test_data = train_test_split(new_data.reset_index(drop = True), test_size = .2, random_state = 42, shuffle = False) #Reset index
train_data.shape

(3842, 7)

In [18]:
#Convert text data
#Create column that combines all existing columns with text that will be used for prediction
new_data['text_features'] = new_data['genres']+' '+new_data['keywords']+' '+new_data['tagline']+' '+new_data['original_language']

#Use TF_IDF Vectorization on text features
vectorizer = TfidfVectorizer(stop_words = 'english')
tfdif_vectorizer = vectorizer.fit_transform(new_data['text_features'])
print(tfdif_vectorizer)

  (0, 2017)	0.04617966488045821
  (0, 4363)	0.3636337984216591
  (0, 6801)	0.17133481036312548
  (0, 2045)	0.3210654681621908
  (0, 5594)	0.28587081511194573
  (0, 1276)	0.3335509700293637
  (0, 6619)	0.16770224387207283
  (0, 5640)	0.45385556834502166
  (0, 2503)	0.22006080738905745
  (0, 1199)	0.2999942603222045
  (0, 1538)	0.28587081511194573
  (0, 2307)	0.13849945267466546
  (0, 5290)	0.13849945267466546
  (0, 2234)	0.14844742341686887
  (0, 187)	0.12055331158850609
  (0, 152)	0.10503609715995893
  (1, 613)	0.24820442693534756
  (1, 2022)	0.2599641177696163
  (1, 3541)	0.167100794921968
  (1, 3628)	0.1545812294148879
  (1, 1316)	0.2873173634798626
  (1, 6226)	0.3498889349060224
  (1, 3097)	0.28993905681508503
  (1, 1948)	0.28028928179381246
  (1, 3213)	0.23814474651413453
  :	:
  (4799, 1294)	0.0929666464026622
  (4799, 2017)	0.04891021724667227
  (4800, 5476)	0.38369009898275336
  (4800, 6033)	0.37755273521343574
  (4800, 4083)	0.40264630508787685
  (4800, 3187)	0.3466430667981653

In [19]:
#Use cosine similarity to evaluate similarity between different movies
cosine_sim_train = cosine_similarity(tfdif_vectorizer, tfdif_vectorizer)
print(cosine_sim_train)

[[1.         0.09328783 0.03376637 ... 0.00286228 0.00389492 0.00322699]
 [0.09328783 1.         0.05129874 ... 0.03503932 0.00388096 0.00321542]
 [0.03376637 0.05129874 1.         ... 0.00348935 0.00474822 0.00393396]
 ...
 [0.00286228 0.03503932 0.00348935 ... 1.         0.00522769 0.0043312 ]
 [0.00389492 0.00388096 0.00474822 ... 0.00522769 1.         0.0058938 ]
 [0.00322699 0.00321542 0.00393396 ... 0.0043312  0.0058938  1.        ]]


4. **Model Evaluation**

In [34]:
#Get 5 movie reccommendations for each row in test set
def recommend_movies(movie_index, cosine_sim=cosine_sim_train, train_data=train_data, test_data=test_data):
    if movie_index >= len(test_data): #Ensure dataframe lengths match
        print("Movie index is out of bounds.")
        return pd.DataFrame()

    test_movie = test_data.iloc[movie_index] #Locate movie by index
    sim_scores = cosine_sim[:, test_movie.name] #Create array for corresponding cosine similarity
    movie_indices = sorted(range(len(sim_scores)), key=lambda i: sim_scores[i], reverse=True) #Sort in descending order
    movie_indices = [idx for idx in movie_indices if idx in train_data.index] #Filter out invalid indices (troubleshooting)
    movie_indices = movie_indices[:5] #Get top 5 movie recommendations

    try:
        recommendations = train_data.iloc[movie_indices].copy() #Create copy of dataframe for results
        recommendations['Cosine Similarity'] = [sim_scores[idx] for idx in movie_indices] #Add cosine similarity
        return recommendations[['Cosine Similarity', 'genres', 'keywords', 'original_title', 'tagline', 'original_language']] #Return dataframe with results
    except IndexError as e: #Troubleshooting
        print(f"IndexError: {e}")
        return pd.DataFrame()

test_movie_index = 4 #Select random index
test_recommendations = recommend_movies(test_movie_index, cosine_sim_train, train_data, test_data) #Store data
print("Recommended Movies for Test Set:") #Print results
test_recommendations.head(10) #Get top 5 results

Recommended Movies for Test Set:


,Cosine Similarity,genres,keywords,original_title,tagline,original_language
397,0.580420,Comedy Romance,graduation ex husband woman director,It's Complicated,First comes marriage. Then comes divorce. And ...,en
811,0.472082,Comedy Romance,cia airport cat jew orderly,Meet the Parents,First comes love. Then comes the interrogation.,en
1949,0.470388,Drama,independent film,Love Ranch,When It Comes To Love... Everyone Pays A Price.,en
3575,0.427399,Comedy,independent film aftercreditsstinger duringcre...,Clerks II,With No Power Comes No Responsibility,en
3117,0.323498,Comedy Romance,independent film,The Good Guy,,en


In [36]:
#Evaluate test by finding average similarity scores of top recommendation
def calculate_average_cosine(test_data, cosine_sim=cosine_sim_train, train_data=train_data):
    avg_cosine_similarity = 0  #Initialize variable

    for _, test_movie in test_data.iterrows(): #Create loop to iterate through test set
        sim_scores = cosine_sim[:, test_movie.name] #Get similarity scores
        movie_indices = sorted(range(len(sim_scores)), key=lambda i: sim_scores[i], reverse=True) #Sort in descending order
        valid_indices = [idx for idx in movie_indices if idx in train_data.index] #Filter out indices not in training set

        if not valid_indices: #Troubleshooting
            print(f"No valid recommendations for movie index {test_movie.name}")
            continue

        movie_index = valid_indices[0] #Take top recommended movie

        try:
            recommendation = train_data.loc[movie_index] #Retrieve useful features from train set
            avg_cosine_similarity += sim_scores[movie_index] #Accumulator to store sum of similarity scores
        except KeyError as e: #Troubleshooting
            print(f"KeyError: {e}")

    if len(test_data) > 0: #Troubleshooting
        avg_cosine_similarity /= len(test_data) #Calculate average
    return avg_cosine_similarity

average_cosine_similarity = calculate_average_cosine(test_data, cosine_sim_train, train_data) #Store average in new variable
print("Average Cosine Similarity for Test Set:", average_cosine_similarity) #Display results

Average Cosine Similarity for Test Set: 0.43541087154708136
